In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

X = np.genfromtxt('../new_data/X_word2vec.csv', delimiter=',')[1:]
Y = np.genfromtxt('../new_data/Y.csv', delimiter=',')[1:]

In [5]:
X_comp, X_test, Y_comp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
Xtr, Xva, Ytr, Yva = train_test_split(X_comp, Y_comp, test_size=0.2)

In [2]:
from sklearn.model_selection import GridSearchCV

parameters1 = {"learning_rate": [0.01, 0.05, 0.075, 0.1,0.2],
               "n_estimators":[10, 100, 250, 500, 1000, 1250, 1500, 1750]}

parameters2 = {"max_depth":[3,5,8]}

parameters3 = {"min_samples_split": [2,6,10,20,40,60],
               "min_samples_leaf": [1,3,5,7,9]}

parameters4 = {"max_features":["log2","sqrt"]}

parameters5 = {"subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0]}

In [ ]:
import time
clf = GridSearchCV(GradientBoostingClassifier(loss="deviance"), parameters1, cv=5)

print('training start')
starting_time = time.time()
clf.fit(Xtr[:5000], Ytr[:5000])
end_time = time.time()
print("training finished, took {} seconds".format(end_time - starting_time))

training start
